# HANK estimation including household parameters

This notebook replicates the estimation of the medium-scale HANK model from [Ensemble MCMC Sampling for DSGE Models](https://gregorboehl.com/live/dime_mcmc_boehl.pdf). This is the estimation with the smaller grid, but the estimation of the model with the larger grid is exactly the same. Please refer to the original paper for details.

Let's start with a few imports:

In [6]:
import warnings
import pathos
import emcee

import numpy as np
import pandas as pd
import emcwrap as ew
import grgrlib.hanktools as gsj

#from grgrlib import *

# set random seed
np.random.seed(0)

Import the model and get estimation infos from the yaml file. Everything is placed in the ``ressources`` folder relative to this notebook.

In [3]:
# import model and model infos
model = gsj.load_model('ressources/hank2_small.py')
est_info = ew.parse_yaml('ressources/hank2.yaml')

In [7]:

# load data
d0 = pd.read_csv('ressources/BS_data.csv',
                 sep=';', index_col='date', parse_dates=True).dropna()
d0.index = pd.date_range('1973Q1', periods=len(d0.index), freq='Q')


In [14]:
# set data
series = ['GDP', 'Infl', 'FFR', 'Cons_JPT', 'Lab', 'Inv_JPT', 'Wage']
data = d0[series]['1983Q1':'2008Q4']
data = data.to_numpy()
# data = data.join(d1[['top10income', 'top10wealth']]['1983Q1':'2008Q4']).to_numpy()


In [9]:

# calculate inital steady state
st = time.time()
hank_ss, ss, unknowns_ss, targets_ss, hank, unknowns, targets, exogenous = model.dag()
print('Initial SS took %ss.' % (time.time() - st))

Initial SS took 2.2782814502716064s.


In [10]:
# general equilibrium jacobians
st = time.time()
G = hank.solve_jacobian(ss, unknowns, targets, exogenous, T=300)
print('Initial Jacobian took %ss.' % (time.time() - st))


Initial Jacobian took 4.85834527015686s.


In [15]:
# set measurement errors
me_sig = np.std(data, axis=0)*1e-1


In [17]:

# get priors from the yaml
prior = est_info['estimation']['prior']
shocks = est_info['declarations']['shocks']
observables = est_info['declarations']['observables']

# compile priors
frozen_prior, prior_func, bptrans, _, _, prior = gsj.get_prior(prior, shocks, verbose=True)

# Compute jacobian of household block
jac_info = {'unknowns': unknowns, 'targets': targets,
            'exogenous': exogenous, 'T': 300, 'ss': ss}


   adding Z_AR_COEF...
   adding rstar_AR_COEF...
   adding G_AR_COEF...
   adding markup_w_AR_COEF...
   adding markup_AR_COEF...
   adding rinv_shock_AR_COEF...
   adding beta_AR_COEF...
   adding Z_SIG_COEF...
   adding rstar_SIG_COEF...
   adding G_SIG_COEF...
   adding markup_w_SIG_COEF...
   adding markup_SIG_COEF...
   adding rinv_shock_SIG_COEF...
   adding beta_SIG_COEF...
Adding parameters to the prior distribution...
   - sig_c as normal with mean 1.5 and std/df 0.375
   - sig_l as normal with mean 2.0 and std/df 0.75
   - chi0 as gamma with mean 0.2 and std/df 0.15
   - tau as beta with mean 0.2 and std/df 0.1
   - sigma_z as normal with mean 1.0 and std/df 0.4
   - phiss as gamma with mean 4.0 and std/df 2.0
   - zeta_p as beta with mean 0.5 and std/df 0.1
   - zeta_w as beta with mean 0.5 and std/df 0.1
   - iota_p as beta with mean 0.5 and std/df 0.15
   - iota_w as beta with mean 0.5 and std/df 0.15
   - phi_pi as gamma with mean 1.5 and std/df 0.25
   - phi_y as gamma 

In [21]:

def data_func(x, ss, data):
    # Remove intercept from series

    data_adj = np.empty_like(data)
    data_adj[:, 0] = data[:, 0] - ss['ybar']  # y
    data_adj[:, 1] = data[:, 1] - ss['pistar']  # pi
    data_adj[:, 2] = data[:, 2] - ss['rstar']  # i
    data_adj[:, 3] = data[:, 3] - ss['ybar']  # c
    data_adj[:, 4] = data[:, 4] - ss['n_obs']  # n
    data_adj[:, 5] = data[:, 5] - ss['ybar']  # I
    data_adj[:, 6] = data[:, 6] - ss['ybar']  # w
    # data_adj[:, 7] = data[:, 7] - ss['top10y_obs']  # w
    # data_adj[:, 8] = data[:, 8] - ss['top10w_obs']  # w

    return data_adj



In [22]:


def ss_func(ss, x):

    ss['pi'] = ss['pistar']/100
    ss['i'] = ss['rstar']/100
    ss['r'] = (1 + ss['i']) / (1 + ss['pi']) - 1

    # the actual function to calculate the steady state
    ss = hank_ss.solve_steady_state(ss, unknowns_ss, targets_ss, solver="hybr")
    ss = hank.steady_state(ss)

    return ss


In [23]:

def log_ll(x):

    # a single likelihood evaluation
    x = bptrans(x)
    x = np.array(x)

    # check prior first, and exit already if infinity
    lprior = prior_func(x)
    if np.isinf(lprior):
        return -np.inf

    st = time.time()
    # calculate likelihood
    llike, ss_local = gsj.get_ll(x, hank, data, data_func, me_sig, jac_info, list(prior), observables, shocks, ss_func=lambda ss, x: ss_func(ss, x), debug=False)

    # store relevant steady state values as initial guess
    if ss_local is None:
        return -np.inf

    # print(llike + lprior, np.round(- st + time.time(),5))

    return llike + lprior



In [19]:
# TODO put all parameters at same place
# set some parameters
ncores = pathos.helpers.cpu_count()
nchain = ncores*4
nsteps = 2000

move = ew.DIMEMove(aimh_prob=0.1)
backend_name = '/home/gboehl/npz/hank_small1.h5'

backend = emcee.backends.HDFBackend(backend_name)


In [24]:
warnings.filterwarnings('ignore')
pool = pathos.pools.ProcessPool()

# get a prior sample
p0 = ew.get_prior_sample(frozen_prior, nchain, mapper=pool.uimap, check_func=lambda x: log_ll(bptrans(x, False)), debug=False)
p0pspace = bptrans(p0, False)

# sample the sampler
sampler = ew.run_mcmc(log_ll, p0=p0pspace, nsteps=nsteps, moves=move, tune=500, priors=prior, prior_transform=bptrans, backend=backend, update_freq=100, pool=pool, maintenance_interval=10)

# old_chain = backend.get_chain()
# sampler = ew.run_mcmc(log_ll, p0=None, nsteps=nsteps-old_chain.shape[0], moves=move, tune=500, priors=prior, backend=backend, update_freq=100, pool=pool, maintenance_interval=10, resume=True)

ew.save_to_backend(sampler, {'tune': 500, 'priors': list(prior)})


100%|██████████| 32/32 [01:32<00:00,  2.90s/it]


(prior_sample:) Sampling done. Check fails for 27.27% of the prior.


  0%|          | 0/2000 [00:00<?, ?sample(s)/s]

RuntimeError: It is unadvisable to use a red-blue move with fewer walkers than twice the number of dimensions.